In [22]:
%pip install --upgrade pymupdf PyPDF2

  Obtaining dependency information for pymupdf from https://files.pythonhosted.org/packages/f0/6d/8b646ba3096cd5ccbd9714c367f9faca18fca072eed83f5622050dbbfde3/PyMuPDF-1.23.4-cp311-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   - -------------------------------------- 0.1/3.5 MB 2.4 MB/s eta 0:00:02
   ----------- ---------------------------- 1.0/3.5 MB 10.9 MB/s eta 0:00:01
   ------------ --------------------------- 1.0/3.5 MB 11.0 MB/s eta 0:00:01
   ------------ --------------------------- 1.0/3.5 MB 11.0 MB/s eta 0:00:01
   ------------------------ --------------- 2.1/3.5 MB 9.1 MB/s eta 0:00:01
   ---------------------------------------  3.5/3.5 MB 12.9 MB/s eta 0:00:01
   ---------------------------------------- 3.5/3.5 MB 12.2 MB/s eta 0:00:00
  Attempting uninstall: pymupdf
    Found existing installation: PyMuPDF 1.23.3
    Uninstalling PyMuPDF-1.23.3:
      Successfully uninstalled PyMuPDF-1.23.3
Note: you may need to resta

ERROR: Could not install packages due to an OSError: [WinError 5] Acesso negado: 'C:\\Users\\msart\\AppData\\Local\\Temp\\pip-uninstall-fc_rmfjc\\_fitz.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [1]:
from PyPDF2 import PdfReader
import re,fitz
import pandas as pd
import os

In [2]:
ocr = False
if ocr:
    from tempfile import TemporaryDirectory
    import pytesseract
    from pdf2image import convert_from_path
    from PIL import Image

In [3]:
#CHANGE FILENAME TO THE NEW ONE
FILENAME = 'ProvasSemImagem/ENEM_2021_P1_CAD_11_DIA_2_LARANJA_LEDOR.pdf'
OUTNAME = 'Data/ENEM_2021_P1_CAD_11_DIA_2_LARANJA_LEDOR.csv'

In [4]:
class EnemAutomata:
    def __init__(self):
        self.state = 0
        self.question = {}
        self.state_dict = {
            0:'header',
            1:'body',
            'A':'alternative A',
            'B':'alternative B',
            'C':'alternative C',
            'D':'alternative D',
            'E':'alternative E',
            'R':'Essay'
        }
    def clear_memory(self):
        self.state = 0
        self.question = {}
        
    def letter_state(self,current_state,next_state,part):
        if part.strip() != next_state:
            if current_state not in self.question:
                self.question[current_state] =''
            self.question[current_state]+=(part.strip('\n')+' ').replace('  ',' ')
        else:
            self.state = next_state
        
    def read(self,part):
        #final state
        if self.state=='E' and (re.search('questão [\d]+',part.lower()) or re.search('ENDOFENEM',part) 
                              or re.search('LC -',part) or re.search('CH -',part) or
                              re.search('\*.*\*',part) or re.search('instruções para a redação',part.lower())):
            ret = self.question.copy()
            self.clear_memory()
            return ret
        elif re.search('\*.*\*',part.lower()) and self.state == 'R':
            ret = self.question.copy()
            self.clear_memory()
            return ret
        
        if not part:
            return False
        elif re.search('questão [\d]+',part.lower()) and self.state == 0:
            self.question['question'] = part.strip()
            self.state = 1
        elif (re.search('instruções para a redação',part.lower()) or re.search('PROPOSTA DE REDAÇÃO',part)) and self.state == 0:
            self.question['question'] = 'redação'
            self.question['body'] = ''
            self.question['A'] = ''
            self.question['B'] = ''
            self.question['C'] = ''
            self.question['D'] = ''
            self.question['E'] = ''
            self.state = 'R'
        elif self.state =='R':
            self.question['body']+=(part.strip('\n')+' ').replace('  ',' ')
        elif self.state == 1 and part.strip() != 'A':
            if 'body' not in self.question:
                self.question['body'] = ''
            self.question['body']+=(part.strip('\n')+' ').replace('  ',' ')
        elif self.state==1:
            self.state = 'A'
        elif self.state != 0 and self.state != 1:
            self.letter_state(self.state,chr(ord(self.state)+1),part)
        return False            

In [5]:
class OCRAutomata:
    def __init__(self):
        self.state = 0
        self.question = {}
        self.state_dict = {
            0:'header',
            1:'body',
            'A':'alternative A',
            'B':'alternative B',
            'C':'alternative C',
            'D':'alternative D',
            'E':'alternative E',
            'R':'Essay'
        }
    def clear_memory(self):
        self.state = 0
        self.question = {}
        self.question['question'] = ''
        self.question['body']=''
        self.question['A'] = None
        self.question['B'] = None
        self.question['C'] = None
        self.question['D'] = None
        self.question['E'] = None
        
    def read(self,part):
        #final state
        if self.state==1 and (re.search('questão [\d]+',part.lower()) or re.search('ENDOFENEM',part) 
                              or re.search('LC -',part) or re.search('CH -',part) or
                              re.search('\*.*\*',part) or re.search('instruções para a redação',part.lower())):
            ret = self.question.copy()
            self.clear_memory()
            return ret
        elif re.search('\*.*\*',part.lower()) and self.state == 'R':
            ret = self.question.copy()
            self.clear_memory()
            return ret
        if not part:
            return False
        elif re.search('questão [\d]+',part.lower()) and self.state == 0:
            self.question['question'] = re.search('questão [\d]+',part.lower()).group()
            self.state = 1
        elif (re.search('instruções para a redação',part.lower()) or re.search('PROPOSTA DE REDAÇÃO',part)) and self.state == 0:
            self.question['question'] = 'redação'
            self.question['body'] = ''
            self.question['A'] = None
            self.question['B'] = None
            self.question['C'] = None
            self.question['D'] = None
            self.question['E'] = None
            self.state = 'R'
        elif self.state =='R':
            self.question['body']+=(part.strip('\n')+' ').replace('  ',' ')
        elif self.state == 1:
            if 'body' not in self.question:
                self.question['body'] = ''
            self.question['body']+=(part.strip('\n')+' ').replace('  ',' ')
        return False            

In [6]:
class PhysicalEnemParser:
    def __init__(self,enem_object,engine='pypdf2'):
        self.enem_object = enem_object
        self.engine=engine
        parts = []
        if engine=='pymupdf':
            for page_num in range(1,len(enem_object)):
                page = enem_object[page_num]
                image_list = page.get_images(full=True)
                to_remove = []
                for image in image_list:
                    bbox = page.get_image_bbox(image)
                    tb = page.get_textbox(bbox)
                    to_remove.extend(tb.split('\n'))
                page_text = page.get_text().split('\n')
                for text in page_text:
                    if text not in to_remove:
                        parts.append(text)
        if engine =='pypdf2':
            def visitor_body(text, cm, tm, fontDict, fontSize):
                parts.append(text)

            for page in enem_object.pages:
                page.extract_text(visitor_text=visitor_body)
        #LINUX only
        if engine =='OCR':
            language_config = r'-l por --psm 1'
            # Path of the Input pdf
            PDF_file = enem_object
            
            # Store all the pages of the PDF in a variable
            image_file_list = []
            
            with TemporaryDirectory() as tempdir:
                # Create a temporary directory to hold our temporary images.
                pdf_pages = convert_from_path(PDF_file, 500)
                
                # Iterate through all the pages stored above
                for page_enumeration, page in enumerate(pdf_pages, start=1):        
                    # Create a file name to store the image
                    filename = f"{tempdir}\page_{page_enumeration:03}.jpg"
                    # Save the image of the page in system
                    page.save(filename, "JPEG")
                    image_file_list.append(filename)
                parsed = ''
                # Iterate from 1 to total number of pages
                for image_file in image_file_list:
                # Recognize the text as string in image using pytesserct
                    text = str(((pytesseract.image_to_string(Image.open(image_file),config=language_config))))
                    parsed+=text
                parts = parsed.split('\n')
        self.parts=parts

    def parse_questions(self):
        self.automata = EnemAutomata()
        if self.engine=='OCR':
            self.automata = OCRAutomata()
        questions=[]
        for part in self.parts:
            accept = self.automata.read(part)
            while accept:
                questions.append(accept)
                accept = self.automata.read(part)
        return questions

In [84]:
files = list(os.listdir('ProvasPre2016'))
files = [f for f in files if '2011' in f]
for fileno in files[0:1]:
    FILENAME = 'ProvasPre2016/'+fileno
    OUTNAME = 'Data/'+fileno.strip('.pdf')+'.csv'
    enem = PdfReader(FILENAME)
    parser = PhysicalEnemParser(enem,engine='pypdf2')
    questions = parser.parse_questions()
    df = pd.DataFrame(questions)
    essay_instructions = df.loc[df['question']=='redação','body']
    if not essay_instructions.empty:
        essay_instructions = essay_instructions.iloc[0]
        try:
            df.loc[df['question']=='redação','body'] = re.sub(r'PROPOSTA DE REDAÇÃO.*?(?!(TEXTO))','',essay_instructions) + re.search(r'PROPOSTA DE REDAÇÃO.*?(?=TEXTO)',essay_instructions).group(0)
        except:
            pass
    for column in df.columns:
        df[column] = df[column].apply(lambda x: x.replace('\t',' ').replace('  ',' ').strip())
    df.to_csv(OUTNAME,index=False)

In [101]:
questions = df['question'].apply(lambda x: None if not re.search('[\d]+',x) else re.search('[\d]+',x).group())

In [103]:
for i in range(91,180):
    if str(i) not in questions.tolist():
        print(i)

In [43]:
questions

0     01
1     02
2     03
3     04
4     05
      ..
83    83
84    87
85    85
86    90
87    88
Name: question, Length: 88, dtype: object

In [47]:
print(parser.parts)

['', '', '', '', '3 ', 'BRANCO', '', 'EXAME NACIONAL DO ENSINO MÉDIO', '', '\n', '1º DIA\n', 'CADERNO', '', '', '\n', '2011', '', 'A COR DO SEU CADERNO DE QUESTÕES É BRANCA .\n', 'MARQUE-A EM SEU CARTÃO -RESPOST A', '', '\n', 'PROVA DE CIÊNCIAS HUMANAS E SU AS TECNOL OGIAS\n', 'PROVA DE CIÊNCIAS DA NA TUREZA E SU AS TECNOL OGIAS', '', '\n', 'LEIA A TENT AMENTE AS INSTRUÇÕES SEGUINTES', '', '\n', '1', 'Este CADERNO DE QUESTÕES contém 90 questões \n', 'numeradas de 1 a 90, dispostas da seguinte maneira:\n', 'a.  as questões de número 1 a 45 são relativas à área \n', 'de Ciências Humanas e suas Tecnologias;\n', 'b. as questões de número 46 a 90 são relativas à \n', 'área de Ciências da Natureza e suas Tecnologias.\n', '2', ' Confira se o seu CADERNO DE QUESTÕES contém a \n', 'quantidade de questões e se essas questões estão na \n', 'ordem mencionada na instrução anterior . Caso o caderno \n', 'esteja incompleto, tenha qualquer defeito ou apresente \n', 'divergência, comunique ao aplicador

In [55]:
E = EnemAutomata()

In [58]:
parts = parser.parts[-136:]

In [78]:
questions=[]
for part in parts:
    print(part)
    accept = E.read(part)
    while accept:
        questions.append(accept)
        accept = E.read(part)

QUESTÃO 89 


Certas espécies de algas são capazes de absorver 

rapidamente compostos inorgânicos presentes na 

água, acumulando-os durante seu crescimento. Essa capacidade fez com que se pensasse em usá-las 


como biofiltros para a limpeza de ambientes aquáticos 


contaminados, removendo, por exemplo, nitrogênio e fósforo de resíduos orgânicos e metais pesados 

provenientes de rejeitos industriais lançados nas 

águas. Na técnica do cultivo integrado, animais e algas crescem de forma associada, promovendo um maior equilíbrio ecológico.  

A utilização da técnica do cultivo integrado de animais 

e algas representa uma proposta favorável a um ecossistema mais equilibrado porque






*LEBR75SAB30*









CN - 1º dia | Caderno 3 - BRANCO - Página 30
























































CN - 1º dia | Caderno 3 - BRANCO - Página 31


*LEBR75SAB31*


















































In [75]:
E = EnemAutomata()
E.read(parts[0])

False

In [76]:
E.state

1

In [79]:
E.question

{'question': 'QUESTÃO 89',
 'body': 'QUESTÃO 89  Certas espécies de algas são capazes de absorver rapidamente compostos inorgânicos presentes na água, acumulando-os durante seu crescimento. Essa capacidade fez com que se pensasse em usá-las  como biofiltros para a limpeza de ambientes aquáticos  contaminados, removendo, por exemplo, nitrogênio e fósforo de resíduos orgânicos e metais pesados provenientes de rejeitos industriais lançados nas águas. Na técnica do cultivo integrado, animais e algas crescem de forma associada, promovendo um maior equilíbrio ecológico.  A utilização da técnica do cultivo integrado de animais e algas representa uma proposta favorável a um ecossistema mais equilibrado porque *LEBR75SAB30*  CN - 1º dia | Caderno 3 - BRANCO - Página 30  CN - 1º dia | Caderno 3 - BRANCO - Página 31 *LEBR75SAB31* '}

In [100]:
df = pd.read_csv('Data/ia2_caderno6_cinza_ledor_2011.csv')